In [13]:
from dataclasses import dataclass
from pathlib import  Path

@dataclass(frozen= True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path


In [14]:
from text_summarizer.constants import *
from text_summarizer.utils.comon import read_yaml, create_directories
import os

class ConfigurationManager:
    def __init__(
            self,
            config_file_path = PROJECT_DIR.joinpath(CONFIG_FILE_PATH),
            params_file_path = PROJECT_DIR.joinpath(PARAMS_FILE_PATH)):
        
        print( read_yaml(config_file_path))
        # print( read_yaml(os.path("../config/config.yaml")))
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([PROJECT_DIR.joinpath(self.config.artifacts_root)])

    def get_model_evaluation_config(self):
        config = self.config.model_evaluation

        create_directories([PROJECT_DIR.joinpath(self.config.root_dir)])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir = PROJECT_DIR.joinpath(config.root_dir),
            artifacts_root = PROJECT_DIR.joinpath(config.artifacts_root),
            model_name = PROJECT_DIR.joinpath(config.model_name),
            model_type = PROJECT_DIR.joinpath(config.model_type),
            model_version = PROJECT_DIR.joinpath(config.model_version),
        )
        return model_evaluation_config


In [15]:
from tqdm import tqdm
import pandas as pd
import torch
from datasets import load_dataset, load_from_disk, load_metric
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [16]:
class ModelEvaluation:
    def __inti__(self,config: ModelEvaluationConfig):
        self.config = config
    
    def generate_batch_sized_chunks(self,list_of_elements,batch_size):
        """split the dataset into smaller batches that can be processed simultaneously
        Yield successive batch-sized chunks from a list of elements."""
        for i in range(0,len(list_of_elements),batch_size):
            yield list_of_elements[i:i+batch_size]
    
    def calculate_metric_on_test_dataset(self,dataset,metric,model,tokenizer,
                                         batch_size=16,device="cuda" if torch.cuda.is_available() else "cpu",
                                         column_text="article",column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text],batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary],batch_size))
        
        for article_batch, target_batch in tqdm(
                zip(article_batches,target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch,max_length=1024, trucation=True,padding="max_length",
                               return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                       attention_mask=inputs["attention_mask"].to(device),
                                       length_penalty=0.8, num_beams=8, max_length = 128)

            decoded_summaries = [tokenizer.decode(summary,skip_special_tokens=True,clean_up_tokenization=True)
                                 for summary in summaries]
            decoded_summaries = [(d.replace(""," ")) for d in decoded_summaries ]   

            metric.add_batch(predictions = decoded_summaries, references=target_batch)

        # Compute and return ROUGE score
        score = metric.compute()
        return score
    
    def evaluate(self):
        device="cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_name = ["rouge1", "rouge2", "rougeL", "rougeLsum" ]

        rouge_metric = load_metric('rouge')
        score = self.calculate_metric_on_test_dataset(
            dataset_samsum_pt['test'][0:10], rouge_metric,model_pegasus,tokenizer,
            batch_size=2,column_text='dialogue', column_summary= 'summary'
        )


        rouge_dict = dict((rn,score[rn].mid.fmeasure  ) for rn in rouge_name)

        df = pd.DataFrame(rouge_dict , index = ['pegasus'])
        df.to_csv(self.config.metric_file_name , index = False)




In [17]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config= model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception  as e:
    raise e

[2023-06-15 23:02:51,422: text_summarizer_logger - INFO:  comon: line 33, yaml file: G:\currentProjects\AI\text_summarizer\config\config.yaml loaded successfully]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'data_path': 'artifacts/data_transformation/samsum_dataset', 'model_ckpt': 'google/pegasus-cnn_dailymail'}}
[2023-06-15 23:02:51,439: text_summarize

BoxKeyError: "'ConfigBox' object has no attribute 'model_evaluation'"

In [20]:

#  print( read_yaml( PROJECT_DIR.joinpath(CONFIG_FILE_PATH)))
import yaml

# with open("../config/config.yaml", 'r') as file:
with open( PROJECT_DIR.joinpath(CONFIG_FILE_PATH), 'r') as file:
    data = yaml.safe_load(file)
PROJECT_DIR.joinpath(CONFIG_FILE_PATH)

WindowsPath('G:/currentProjects/AI/text_summarizer/config/config.yaml')